<a href="https://colab.research.google.com/github/NIXBLACK11/ModelForAnalysis/blob/main/analyseFunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from PIL import Image

audioIndex = 0
imageIndex = 0
totalTime = 0

# Load the 'adamax' model
optimizer_name = 'adamax'
model_path = '/content/drive/MyDrive/videos/video_analysis_vgg16_adamax.h5'
model = load_model(model_path)

# Define the target size expected by the model
target_size = (224, 224)

def preprocess_image(img):
    # Resize the image to the target size
    img = img.resize(target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the pixel values to be between 0 and 1
    return img_array

def predict_genre(img):
    img_array = preprocess_image(img)
    prediction = model.predict(img_array)
    predicted_genre = genres[np.argmax(prediction)]
    return predicted_genre

def extract_screenshots_and_audio(input_video_path, output_screenshots_folder, output_audio_folder):
    global audioIndex, imageIndex, totalTime

    # Create output folders if they don't exist
    os.makedirs(output_screenshots_folder, exist_ok=True)
    os.makedirs(output_audio_folder, exist_ok=True)

    # Load video
    video_clip = VideoFileClip(input_video_path)

    # Extract screenshots and calculate mean time duration
    screenshots = []
    mean_time_duration = 0
    for timestamp in range(0, int(video_clip.duration), 2):
        screenshot = video_clip.get_frame(timestamp)

        # Convert NumPy array to PIL Image
        pil_image = Image.fromarray(screenshot)

        # Predict the genre using the 'adamax' model
        predicted_genre = predict_genre(pil_image)

        # Save screenshot with predicted genre as the filename
        screenshot_path = os.path.join(output_screenshots_folder, f"{predicted_genre}_{imageIndex}.png")
        imageIndex += 1
        pil_image.save(screenshot_path)

        screenshots.append(pil_image)
        mean_time_duration += timestamp

    totalTime += video_clip.duration

    # Extract audio in 30-second chunks
    audio = video_clip.audio
    chunk_duration = 2  # seconds

    for start_time in range(0, int(video_clip.duration), chunk_duration):
        end_time = min(start_time + chunk_duration, int(video_clip.duration))
        audio_chunk = audio.subclip(start_time, end_time)
        audio_chunk_path = os.path.join(output_audio_folder, f"audio_{audioIndex}_{start_time}-{end_time}.wav")
        audio_chunk.write_audiofile(audio_chunk_path, codec='pcm_s16le', ffmpeg_params=["-ac", "1"])
        audioIndex += 1

    # Close video clip
    video_clip.close()

if __name__ == "__main__":
    input_video_path = "/content/drive/MyDrive/videos/TestVideos/GamingType.mp4"
    output_screenshots_folder = "/content/Test/screenshot"
    output_audio_folder = "/content/Test/audio"

    genres = ['MrBeastType', 'VlogType', 'TechReviewType', 'GamingType', 'MinimalistType']

    # Call the function with the video file path
    extract_screenshots_and_audio(input_video_path, output_screenshots_folder, output_audio_folder)

    # Calculate average time (if needed)
    averageTime = totalTime
    print(averageTime)


import os
import numpy as np
from keras.preprocessing import image
from keras.models import load_model

# Function to preprocess an image for prediction
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the pixel values to be between 0 and 1
    return img_array, img

# Function to predict the genre using the 'adamax' model
def predict_genre(img):
    # Load the 'adamax' model
    optimizer_name = 'adamax'
    model_path = f'/content/drive/MyDrive/videos/video_analysis_vgg16_{optimizer_name}.h5'
    model = load_model(model_path)

    # Make a prediction
    prediction = model.predict(img)

    # Get the genre with the highest probability
    predicted_genre = genres[np.argmax(prediction)]
    return predicted_genre

# Define genre folders
genre_folders = {
    'MrBeastType': 'MrBeastType',
    'VlogType': 'VlogType',
    'TechReviewType': 'TechReviewType',
    'GamingType': 'GamingType',
    'MinimalistType': 'MinimalistType'
}

# Initialize counters for each genre
genre_counts = {genre: 0 for genre in genre_folders}

# Loop through all screenshots
for image_name in os.listdir("/content/Test/screenshot"):
    # Preprocess the image
    image_path = os.path.join("/content/Test/screenshot", image_name)
    img, original_img = preprocess_image(image_path)

    # Predict the genre using the 'adamax' model
    predicted_genre = predict_genre(img)
    print(predicted_genre)
    # Update counters for each genre
    genre_counts[predicted_genre] += 1

# Print the overall genre based on majority vote
print(genre_counts)
overall_genre = max(genre_counts, key=genre_counts.get)
print(f"Overall Genre: {overall_genre}")


import os
import librosa
import numpy as np
from keras.models import load_model

# Function to extract MFCC features from audio files
def extract_features(file_path):
    y, sr = librosa.load(file_path, mono=True, duration=5)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfccs

# Load the trained model
model_path = '/content/drive/MyDrive/videos/audio_analysis_multi_genre.h5'  # Use the model trained for multiple genres
model = load_model(model_path)

# Audio folder path
audio_folder = '/content/Test/audio'

# Initialize counters for each genre
genre_counts = {genre: 0 for genre in genres}

# Loop through all audio files in the folder
for audio_file in os.listdir(audio_folder):
    # Construct the full path for each audio file
    audio_path = os.path.join(audio_folder, audio_file)

    # Extract features from the audio file
    audio_features = extract_features(audio_path)

    # Ensure the features have the expected shape (216 time steps)
    expected_shape = (216, 13)
    if audio_features.shape[1] < expected_shape[0]:
        # Pad the features if they have fewer time steps
        audio_features = np.pad(audio_features, ((0, 0), (0, expected_shape[0] - audio_features.shape[1])))
    elif audio_features.shape[1] > expected_shape[0]:
        # Truncate the features if they have more time steps
        audio_features = audio_features[:, :expected_shape[0]]

    # Transpose and reshape the features
    audio_features = audio_features.transpose(1, 0)  # Transpose
    audio_features = audio_features.reshape(1, audio_features.shape[0], audio_features.shape[1])

    # Predict the class
    prediction = model.predict(audio_features)

    # Map the predicted class to a label
    predicted_class = genres[np.argmax(prediction)]

    # Update counters for each genre
    genre_counts[predicted_class] += 1

# Determine the overall predicted genre based on majority vote
overall_genre = max(genre_counts, key=genre_counts.get)

# Print the prediction for each part and the overall predicted genre
print("Predictions for Each Part:")
for genre, count in genre_counts.items():
    print(f"{genre}: {count} parts")

print(f"\nOverall Predicted Genre: {overall_genre}")


1/1 [==============================] - 1s 538ms/step
MoviePy - Writing audio in /content/Test/audio/audio_0_0-2.wav


MoviePy - Done.
MoviePy - Writing audio in /content/Test/audio/audio_1_2-4.wav


MoviePy - Done.
MoviePy - Writing audio in /content/Test/audio/audio_2_4-6.wav


MoviePy - Done.
MoviePy - Writing audio in /content/Test/audio/audio_3_6-8.wav


MoviePy - Done.
MoviePy - Writing audio in /content/Test/audio/audio_4_8-10.wav


...

MoviePy - Done.
MoviePy - Writing audio in /content/Test/audio/audio_847_1694-1696.wav


MoviePy - Done.
MoviePy - Writing audio in /content/Test/audio/audio_848_1696-1698.wav


MoviePy - Done.
1698.28
1/1 [==============================] - 1s 1s/step
MrBeastType
1/1 [==============================] - 1s 1s/step
MrBeastType
1/1 [==============================] - 1s 1s/step
MrBeastType


1/1 [==============================] - 1s 683ms/step
GamingType


1/1 [==============================] - 1s 671ms/step
TechReviewType
1/1 [==============================] - 1s 695ms/step
GamingType
1/1 [==============================] - 1s 783ms/step
MinimalistType
1/1 [==============================] - 1s 1s/step
GamingType
1/1 [==============================] - 1s 705ms/step
MinimalistType
1/1 [==============================] - 1s 707ms/step
MrBeastType
1/1 [==============================] - 1s 680ms/step
GamingType
1/1 [==============================] - 1s 691ms/step
TechReviewType
1/1 [==============================] - 1s 717ms/step
MinimalistType
1/1 [==============================] - 1s 1s/step
GamingType
1/1 [==============================] - 1s 1s/step
MrBeastType
1/1 [==============================] - 1s 1s/step
GamingType
1/1 [==============================] - 1s 692ms/step
MrBeastType
1/1 [==============================] - 1s 681ms/step
GamingType
1/1 [==============================] - 1s 702ms/step
MrBeastType
1/1 [=======================

In [7]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from PIL import Image
import librosa

# Hard-coded paths
input_video_path = "/content/drive/MyDrive/videos/TestVideos/GamingType.mp4"
output_screenshots_folder = "/content/Test/screenshot"
output_audio_folder = "/content/Test/audio"
model_image_path = '/content/drive/MyDrive/videos/video_analysis_vgg16_adamax.h5'
model_image = load_model(model_image_path)#add
model_audio_path = '/content/drive/MyDrive/videos/audio_analysis_multi_genre.h5'
model_audio = load_model(model_audio_path)
audioIndex = 0
imageIndex = 0
totalTime = 0
genres = ['MrBeastType', 'VlogType', 'TechReviewType', 'GamingType', 'MinimalistType']

def preprocess_image(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def extract_features(file_path):
    y, sr = librosa.load(file_path, mono=True, duration=5)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfccs

def predict_genre(img):
    # # Load the 'adamax' model
    # optimizer_name = 'adamax'
    # model_image_path = f'/content/drive/MyDrive/videos/video_analysis_vgg16_{optimizer_name}.h5'
    # model_image = load_model(model_image_path)

    img_array = preprocess_image(img)
    prediction = model_image.predict(img_array)
    predicted_genre = genres[np.argmax(prediction)]
    return predicted_genre


def extract_screenshots_and_audio(input_video_path, output_screenshots_folder, output_audio_folder):
    global audioIndex, imageIndex, totalTime

    # Create output folders if they don't exist
    os.makedirs(output_screenshots_folder, exist_ok=True)
    os.makedirs(output_audio_folder, exist_ok=True)

    # Load video
    video_clip = VideoFileClip(input_video_path)

    # # Extract screenshots and calculate mean time duration
    # screenshots = []
    # mean_time_duration = 0
    # for timestamp in range(0, int(video_clip.duration), 2):
    #     screenshot = video_clip.get_frame(timestamp)

    #     # Convert NumPy array to PIL Image
    #     pil_image = Image.fromarray(screenshot)

    #     # Predict the genre using the 'adamax' model
    #     predicted_genre = predict_genre(pil_image)

    #     # Save screenshot with predicted genre as the filename
    #     screenshot_path = os.path.join(output_screenshots_folder, f"{predicted_genre}_{imageIndex}.png")
    #     imageIndex += 1
    #     pil_image.save(screenshot_path)

    #     screenshots.append(pil_image)
    #     mean_time_duration += timestamp

    # totalTime += video_clip.duration

    # Extract audio in 30-second chunks
    audio = video_clip.audio
    chunk_duration = 2  # seconds

    for start_time in range(0, int(video_clip.duration), chunk_duration):
        end_time = min(start_time + chunk_duration, int(video_clip.duration))
        audio_chunk = audio.subclip(start_time, end_time)
        audio_chunk_path = os.path.join(output_audio_folder, f"audio_{audioIndex}_{start_time}-{end_time}.wav")
        audio_chunk.write_audiofile(audio_chunk_path, codec='pcm_s16le', ffmpeg_params=["-ac", "1"])
        audioIndex += 1

    # Close video clip
    video_clip.close()

def analyseVideo(video_path):
    # Extract screenshots and audio
    extract_screenshots_and_audio(video_path, output_screenshots_folder, output_audio_folder)

    Predict genre from screenshots
    genre_counts = {genre: 0 for genre in genres}
    for image_name in os.listdir(output_screenshots_folder):
        image_path = os.path.join(output_screenshots_folder, image_name)
        img = Image.open(image_path)
        predicted_genre = predict_genre(img)
        genre_counts[predicted_genre] += 1

    # Print genre predictions from screenshots
    print("Predictions for Each Screenshot:")
    for genre, count in genre_counts.items():
        print(f"{genre}: {count} screenshots")

    # Predict genre from audio
    genre_counts_audio = {genre: 0 for genre in genres}
    for audio_file in os.listdir(output_audio_folder):
        audio_path = os.path.join(output_audio_folder, audio_file)
        audio_features = extract_features(audio_path)

        # Ensure the features have the expected shape (216 time steps)
        expected_shape = (216, 13)
        if audio_features.shape[1] < expected_shape[0]:
            # Pad the features if they have fewer time steps
            audio_features = np.pad(audio_features, ((0, 0), (0, expected_shape[0] - audio_features.shape[1])))
        elif audio_features.shape[1] > expected_shape[0]:
            # Truncate the features if they have more time steps
            audio_features = audio_features[:, :expected_shape[0]]

        # Transpose and reshape the features
        audio_features = audio_features.transpose(1, 0)  # Transpose
        audio_features = audio_features.reshape(1, audio_features.shape[0], audio_features.shape[1])

        audio_features = audio_features[:, :expected_shape[0]]
        prediction_audio = model_audio.predict(audio_features)
        predicted_genre_audio = genres[np.argmax(prediction_audio)]
        genre_counts_audio[predicted_genre_audio] += 1

    # Print genre predictions from audio
    print("\nPredictions for Each Audio Segment:")
    for genre, count in genre_counts_audio.items():
        print(f"{genre}: {count} audio segments")

    # Determine the overall predicted genre based on majority vote from both sources
    overall_genre_screenshot = max(genre_counts, key=genre_counts.get)
    overall_genre_audio = max(genre_counts_audio, key=genre_counts_audio.get)
    print(f"\nOverall Predicted Genre from Screenshots: {overall_genre_screenshot}")
    print(f"Overall Predicted Genre from Audio: {overall_genre_audio}")

# Call the function with the hard-coded video path
analyseVideo("/content/drive/MyDrive/videos/TestVideos/GamingType.mp4")


1/1 [==============================] - 0s 24ms/step


  warnings.warn(



1/1 [==============================] - 0s 36ms/step

Predictions for Each Audio Segment:
MrBeastType: 187 audio segments
VlogType: 0 audio segments
TechReviewType: 1 audio segments
GamingType: 211 audio segments
MinimalistType: 450 audio segments


NameError: name 'genre_counts' is not defined

In [8]:
! rm -rf Test sample_data

In [18]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from PIL import Image
import librosa

output_audio_folder = "/content/Test/audio"
model_image_path = '/content/drive/MyDrive/videos/video_analysis_vgg16_adamax.h5'
model_image = load_model(model_image_path)
model_audio_path = '/content/drive/MyDrive/videos/audio_analysis_multi_genre.h5'
model_audio = load_model(model_audio_path)

audioIndex = 0
imageIndex = 0
totalTime = 0

genres = ['MrBeastType', 'VlogType', 'TechReviewType', 'GamingType', 'MinimalistType']

def preprocess_image(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def predict_genre(img):
    img_array = preprocess_image(img)
    prediction = model_image.predict(img_array)
    predicted_genre = genres[np.argmax(prediction)]
    return predicted_genre

def analyseScreenshot(video_path):
    video_clip = VideoFileClip(video_path)
    genre_counts = {genre: 0 for genre in genres}

    for timestamp in range(0, int(video_clip.duration), 2):
        screenshot = video_clip.get_frame(timestamp)

        pil_image = Image.fromarray(screenshot)

        predicted_genre = predict_genre(pil_image)

        genre_counts[predicted_genre] += 1

    print("Predictions for Each Screenshot:")
    for genre, count in genre_counts.items():
        print(f"{genre}: {count} screenshots")

    video_clip.close()

def extract_features(audio_data, sr):
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=13)
    return mfccs

def analyseAudio(video_path):
    video_clip = VideoFileClip(video_path)
    audio = video_clip.audio
    chunk_duration = 2  # seconds

    genre_counts_audio = {genre: 0 for genre in genres}
    for start_time in range(0, int(video_clip.duration), chunk_duration):
        end_time = min(start_time + chunk_duration, int(video_clip.duration))
        audio_chunk = audio.subclip(start_time, end_time)

        # Extract audio data and sampling rate
        audio_data = audio_chunk.to_soundarray(fps=44100)[:, 0]  # Assuming mono audio
        sr = 44100

        audio_features = extract_features(audio_data, sr)

        # Ensure the features have the expected shape (216 time steps)
        expected_shape = (216, 13)
        if audio_features.shape[1] < expected_shape[0]:
            # Pad the features if they have fewer time steps
            audio_features = np.pad(audio_features, ((0, 0), (0, expected_shape[0] - audio_features.shape[1])))
        elif audio_features.shape[1] > expected_shape[0]:
            # Truncate the features if they have more time steps
            audio_features = audio_features[:, :expected_shape[0]]

        # Transpose and reshape the features
        audio_features = audio_features.transpose(1, 0)  # Transpose
        audio_features = audio_features.reshape(1, audio_features.shape[0], audio_features.shape[1])

        audio_features = audio_features[:, :expected_shape[0]]
        prediction_audio = model_audio.predict(audio_features)
        predicted_genre_audio = genres[np.argmax(prediction_audio)]
        genre_counts_audio[predicted_genre_audio] += 1

    # Print genre predictions from audio
    print("\nPredictions for Each Audio Segment:")
    for genre, count in genre_counts_audio.items():
        print(f"{genre}: {count} audio segments")

    # Close video clip
    video_clip.close()


def analyseVideo(video_path):
    # analyseScreenshot(video_path)
    analyseAudio(video_path)

# Call the function with the hard-coded video path
analyseVideo("/content/drive/MyDrive/videos/TestVideos/GamingType.mp4")


  return stacker(self.iter_chunks(fps=fps, quantize=quantize,



1/1 [==============================] - 0s 35ms/step

Predictions for Each Audio Segment:
MrBeastType: 230 audio segments
VlogType: 0 audio segments
TechReviewType: 1 audio segments
GamingType: 3 audio segments
MinimalistType: 615 audio segments


In [19]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from PIL import Image
import librosa

model_image_path = '/content/drive/MyDrive/videos/video_analysis_vgg16_adamax.h5'
model_image = load_model(model_image_path)
model_audio_path = '/content/drive/MyDrive/videos/audio_analysis_multi_genre.h5'
model_audio = load_model(model_audio_path)

genres = ['MrBeastType', 'VlogType', 'TechReviewType', 'GamingType', 'MinimalistType']

def preprocess_image(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def predict_genre(img):
    img_array = preprocess_image(img)
    prediction = model_image.predict(img_array)
    predicted_genre = genres[np.argmax(prediction)]
    return predicted_genre

def analyseScreenshot(video_path):
    video_clip = VideoFileClip(video_path)
    genre_counts = {genre: 0 for genre in genres}

    for timestamp in range(0, int(video_clip.duration), 10):
        screenshot = video_clip.get_frame(timestamp)

        pil_image = Image.fromarray(screenshot)

        predicted_genre = predict_genre(pil_image)

        genre_counts[predicted_genre] += 1

    print("Predictions for Each Screenshot:")
    for genre, count in genre_counts.items():
        print(f"{genre}: {count} screenshots")

    video_clip.close()

def extract_features(audio_data, sr):
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=13)
    return mfccs

def analyseAudio(video_path):
    video_clip = VideoFileClip(video_path)
    audio = video_clip.audio
    chunk_duration = 10  # seconds

    genre_counts_audio = {genre: 0 for genre in genres}
    for start_time in range(0, int(video_clip.duration), chunk_duration):
        end_time = min(start_time + chunk_duration, int(video_clip.duration))
        audio_chunk = audio.subclip(start_time, end_time)

        audio_data = audio_chunk.to_soundarray(fps=44100)[:, 0]
        sr = 44100

        audio_features = extract_features(audio_data, sr)

        expected_shape = (216, 13)
        if audio_features.shape[1] < expected_shape[0]:
            audio_features = np.pad(audio_features, ((0, 0), (0, expected_shape[0] - audio_features.shape[1])))
        elif audio_features.shape[1] > expected_shape[0]:
            audio_features = audio_features[:, :expected_shape[0]]

        audio_features = audio_features.transpose(1, 0)
        audio_features = audio_features.reshape(1, audio_features.shape[0], audio_features.shape[1])

        audio_features = audio_features[:, :expected_shape[0]]
        prediction_audio = model_audio.predict(audio_features)
        predicted_genre_audio = genres[np.argmax(prediction_audio)]
        genre_counts_audio[predicted_genre_audio] += 1

    print("\nPredictions for Each Audio Segment:")
    for genre, count in genre_counts_audio.items():
        print(f"{genre}: {count} audio segments")

    video_clip.close()


def analyseVideo(video_path):
    analyseScreenshot(video_path)
    analyseAudio(video_path)

analyseVideo("/content/drive/MyDrive/videos/TestVideos/GamingType.mp4")


1/1 [==============================] - 0s 35ms/step
Predictions for Each Screenshot:
MrBeastType: 48 screenshots
VlogType: 14 screenshots
TechReviewType: 13 screenshots
GamingType: 61 screenshots
MinimalistType: 34 screenshots
1/1 [==============================] - 0s 30ms/step

Predictions for Each Audio Segment:
MrBeastType: 28 audio segments
VlogType: 0 audio segments
TechReviewType: 9 audio segments
GamingType: 16 audio segments
MinimalistType: 117 audio segments
